In [66]:
import pandas as pd
import requests
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.svm import SVC
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
import pylab
import matplotlib.pyplot as plt
import re
import seaborn
import random
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [76]:
df1 = pd.read_csv('https://raw.githubusercontent.com/Columbia-Intro-Data-Science/python-introduction-lee-jw/master/data.csv')
slugs = df1['slug']
apikey = "b1d425b38e13174991d87082772805bd5ea3a6dc"
url = "https://api.trackico.io/v1/icos/profile/"

In [80]:
def getInfo(slug):
    proj_url = url + slug + "/?apiKey=" + apikey
    title = requests.get(proj_url).json()["title"]
    slug = requests.get(proj_url).json()["slug"]
    ticker = requests.get(proj_url).json()["ticker"]
    ico_type = requests.get(proj_url).json()["ico_type"]
    rating = requests.get(proj_url).json()["rating"]
    return(title, slug, ticker, ico_type, rating) 

def getInfo2(slug):
    proj_url = url + slug + "/?apiKey=" + apikey
    platform = requests.get(proj_url).json()["platform"]
    accepted_currencies = requests.get(proj_url).json()["finance"]["accepted_currencies"]
    country = requests.get(proj_url).json()["country"]
    category = requests.get(proj_url).json()["category"]
    return(platform, accepted_currencies, country, category)

def getMisc(slug):
    proj_url = url + slug + "/?apiKey=" + apikey    
    web_url = requests.get(proj_url).json()["url"]
    description = requests.get(proj_url).json()["description"]
    link_bitcointalk = requests.get(proj_url).json()["links"]["bitcointalk"]    
    return(web_url, description, link_bitcointalk)
    
def getDates(slug):
    proj_url = url + slug + "/?apiKey=" + apikey
    pre_ico_start = requests.get(proj_url).json()["pre_ico_start"]
    pre_ico_end = requests.get(proj_url).json()["pre_ico_end"]
    ico_start = requests.get(proj_url).json()["ico_start"]
    ico_end = requests.get(proj_url).json()["ico_end"]
    return(pre_ico_start, pre_ico_end, ico_start, ico_end)
    
def getFinances(slug):
    proj_url = url + slug + "/?apiKey=" + apikey
    token_for_sale = requests.get(proj_url).json()["finance"]["token_for_sale"]
    token_supply = requests.get(proj_url).json()["finance"]["token_supply"]
    soft_cap = requests.get(proj_url).json()["finance"]["soft_cap"]
    soft_cap_currency = requests.get(proj_url).json()["finance"]["soft_cap_currency"]
    hard_cap = requests.get(proj_url).json()["finance"]["hard_cap"]
    hard_cap_currency = requests.get(proj_url).json()["finance"]["hard_cap_currency"]
    token_price_in_usd = requests.get(proj_url).json()["finance"]["token_price_in_usd"]
    total_raised_in_usd = requests.get(proj_url).json()["finance"]["total_raised_in_usd"]
    bonus_available = requests.get(proj_url).json()["finance"]["bonus_available"]
    return(token_for_sale, token_supply, soft_cap, soft_cap_currency, hard_cap, hard_cap_currency, token_price_in_usd, total_raised_in_usd, bonus_available)

def getLegal(slug):
    proj_url = url + slug + "/?apiKey=" + apikey
    whitelist_required = requests.get(proj_url).json()["finance"]["whitelist_required"]
    kyc_required = requests.get(proj_url).json()["finance"]["kyc_required"]
    restricted_countries = requests.get(proj_url).json()["finance"]["restricted_countries"]
    return(whitelist_required, kyc_required, restricted_countries)
    
def getInterests(slug):
    proj_url = url + slug + "/?apiKey=" + apikey    
    alexa_global_rank = requests.get(proj_url).json()["interest"]["alexa_global_rank"]
    twitter_participants = requests.get(proj_url).json()["interest"]["twitter_participants"]
    telegram_members = requests.get(proj_url).json()["interest"]["telegram_members"]
    video_views = requests.get(proj_url).json()["interest"]["video_views"]
    return(alexa_global_rank, twitter_participants, telegram_members, video_views)

def getPrice(slug):
    list_url = "https://api.coinmarketcap.com/v2/listings/"
    ticker = getInfo(slug)[1]
    price_data = requests.get(list_url).json()["data"]
    for i in range(len(price_data)-1):
        if price_data[i]['symbol'] == ticker: idx = price_data[i]['id']
    proj_url = "https://api.coinmarketcap.com/v2/ticker/" + str(idx)
    price = requests.get(proj_url).json()["data"]["quotes"]["price"] # can get circ/total/max supplies if needed
    return(price)
print(getInfo('red-pulse'))
print(getInterests('red-pulse'))

#df = pd.DataFrame(columns = list(['title','slug','ticker','ico_type','platform','accepted_currencies','country','category','rating','token_for_sale','token_supply','soft_cap','soft_cap_currency','hard_cap','hard_cap_currency','token_price_in_usd','total_raised_in_usd','bonus_available','whitelist_required','kyc_required','restricted_countries','alexa_global_rank','twitter_participants','telegram_members','video_views']))

#df = pd.DataFrame(columns = list(['title','slug','ticker','ico_type','platform','accepted_currencies','country','category','rating','token_for_sale','token_supply','soft_cap','soft_cap_currency','hard_cap','hard_cap_currency','token_price_in_usd','total_raised_in_usd','bonus_available','whitelist_required','kyc_required','restricted_countries','alexa_global_rank','twitter_participants','telegram_members','video_views']))
#df_y = pd.DataFrame(colums = list(['current_price', 'ROI']))

#for slug in slugs:
#    df = df.append({'title': getInfo(slug)[0], 'slug': getInfo(slug)[1], 'ticker': getInfo(slug)[2], 'ico_type': getInfo(slug)[3], 'platform': getInfo(slug)[4], 'accepted_currencies': getInfo(slug)[5], 'country': getInfo(slug)[6], 'category': getInfo(slug)[7], 'rating': getInfo(slug)[8], 'token_for_sale': getFinances(slug)[0], 'token_supply': getFinances(slug)[1], 'soft_cap': getFinances(slug)[2], 'soft_cap_currency': getFinances(slug)[3], 'hard_cap': getFinances(slug)[4], 'hard_cap_currency': getFinances(slug)[5], 'token_price_in_usd': getFinances(slug)[6], 'total_raised_in_usd': getFinances(slug)[7], 'bonus_available': getFinances(slug)[8], 'whitelist_required': getLegal(slug)[0], 'kyc_required': getLegal(slug)[1], 'restricted_countries': getLegal(slug)[2], 'alexa_global_rank': getInterests(slug)[0], 'twitter_participants': getInterests(slug)[1], 'telegram_members': getInterests(slug)[2], 'video_views': getInterests(slug)[3]}, ignore_index=True)
    #df_y = df.append({'current_price': getPrice(slug), 'ROI': getPrice(slug)/getFinances(slug)[6]}, ignore_index=True)
#df.head()
#df_y.head()
#df.to_csv('data_masterlist.csv')

(u'Red Pulse', u'red-pulse', u'RPX', u'Trading', u'3.3')


ValueError: No JSON object could be decoded

In [56]:
df = pd.DataFrame(columns = list(['title','slug','ticker']))
df.head()
for i in range(5):
    df = df.append({'title': str(i), 'slug': str(i), 'ticker': str(i)}, ignore_index=True)
df.head()
#for i in range(1):
#    df = pd.DataFrame([getData('aphelion'), getData('aphelion'), getData('aphelion'), getData('aphelion')], columns = list(['title','slug','ticker','ico_type']))
#df.head()

<type 'int'>


,title,slug,ticker
0,0,0,0
1,1,1,1
2,2,2,2
3,3,3,3
4,4,4,4


,test1,test2


In [34]:
a = []
for i in range(4):
    a.append(str(i))
    print(a)

['0']
['0', '1']
['0', '1', '2']
['0', '1', '2', '3']


In [5]:

#    print(response.json()["title"])
#    print(response.json()["slug"])
#    print(response.json()["ticker"])
#    print(response.json()["ico_type"])
#    print(response.json()["url"])
#    print(response.json()["description"])
#    print(response.json()["pre_ico_start"])
#    print(response.json()["pre_ico_end"])
#    print(response.json()["ico_start"])
#    print(response.json()["ico_end"])
#    print(response.json()["platform"])
#    print(response.json()["country"])
#    print(response.json()["category"])
#    print(response.json()["rating"])
#    print(response.json()["links"]["bitcointalk"])    
#    print(response.json()["finance"]["token_for_sale"])
#    print(response.json()["finance"]["token_supply"])
#    print(response.json()["finance"]["soft_cap"])
#    print(response.json()["finance"]["soft_cap_currency"])
#    print(response.json()["finance"]["hard_cap"])
#    print(response.json()["finance"]["hard_cap_currency"])
#    print(response.json()["finance"]["token_price_in_usd"])
#    print(response.json()["finance"]["total_raised_in_usd"])
#    print(response.json()["finance"]["whitelist_required"])
#    print(response.json()["finance"]["bonus_available"])
#    print(response.json()["finance"]["kyc_required"])
#    print(response.json()["finance"]["accepted_currencies"])
#    print(response.json()["finance"]["restricted_countries"])
#    print(response.json()["interest"]["alexa_global_rank"])
#    print(response.json()["interest"]["twitter_participants"])
#    print(response.json()["interest"]["telegram_members"])
#    print(response.json()["interest"]["video_views"])
#    try:
#        print(response.json()["market"]["price_usd"])
#        print(response.json()["market"]["available_supply_usd"])
#        print(response.json()["market"]["roi_percent"])
#    except:
#        print('null')
